## Creating a New AutoCNet Server Project

In [1]:
import os
os.environ['autocnet_config'] = '/home/jlaura/autocnet_projects/demo.yml'

import autocnet_server
from autocnet_server.db.model import Base

from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy_utils import database_exists, create_database

This is the code that creates the database and associated tables for an AutoCNet project. No database names or login information is specified in this file.  This is because everything is being stored in a `YAML` configuration file.

In [3]:
!cat '/home/jlaura/autocnet_projects/demo.yml'

### Python Configuration ###
python:
    pyroot: '/usgs/apps/anaconda/envs/autocnet/'
    pybin: '/usgs/apps/anaconda/envs/autocnet/bin/python' # Do not change me


### Cluster Configuration ###
cluster:
    # The number of times to retry a failing cluster job
    maxfailures: 3
    # Which processing queue should be used?
    queue: 'shortall'
    # Location to put <jobid.log> files for cluster jobs
    cluster_log_dir: '/home/jlaura/logs'
    cluster_submission: 'slurm'  # or `pbs`
    tmp_scratch_dir: '/scratch/jlaura'

    # The amount of RAM (in MB) to request for jobs
    extractor_memory: 8192
    processing_memory: 4000

### Database Configuration ###
database:
    database_username: 'jlaura'
    database_password: 'jlaura'
    database_host: 'smalls'
    database_port: 8001
    pgbouncer_port: 8003
    # The name of the database to connect to.  Tables will be created inside this DB.
    database_name: 'jaydemo' # This needs to be all lowercase for PostGreSQL!
    # The number 

In [3]:
config = autocnet_server.config
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(config['database']['database_username'],
                                                            config['database']['database_password'],
                                                            config['database']['database_host'],
                                                            config['database']['database_port'],
                                                            config['database']['database_name']))
if not database_exists(engine.url):
    create_database(engine.url, template='template_postgis')

# Create the tables
Base.metadata.bind = engine
Base.metadata.create_all(engine)